In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
model = malaya.paraphrase.transformer(model = 'small-t5')

2022-11-24 16:26:00.453914: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 16:26:00.468595: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-24 16:26:00.468636: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-11-24 16:26:00.468645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-11-24 16:26:00.468752: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-11-24 16:26:00.468788: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] ke

In [4]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [5]:
import json

test = []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        test.append(json.loads(l))

In [6]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(test), batch_size)):
    input_ids = [s['translation']['src'] for s in test[i:i + batch_size]]
    outputs = model.greedy_decoder(input_ids)
    results.extend(outputs)

  0%|                                                                                                       | 0/5544 [00:00<?, ?it/s]2022-11-24 16:26:05.788787: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
import feature inputs[[445 4435 722 31 4489 2036 158 671 12358 2572 17 10100 10999 358 29476 29756 671 13 1131 8112 342 4603 1416 16 1288 3262 1779 36 5509 11010 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5544/5544 [45:23<00:00,  2.04it/s]


In [7]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(test[no]['translation']['tgt'])

In [8]:
refs = [filtered_right]
sys = filtered_left
r = bleu.corpus_score(sys, refs)
r.__dict__

{'name': 'BLEU',
 'score': 37.24731076156855,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '61.6/41.7/32.5/26.3 (BP = 0.968 ratio = 0.968 hyp_len = 97840 ref_len = 101064)',
 'bp': 0.9675852367370129,
 'counts': [60277, 38474, 28173, 21382],
 'totals': [97840, 92296, 86752, 81208],
 'sys_len': 97840,
 'ref_len': 101064,
 'precisions': [61.60772690106296,
  41.68544682326428,
  32.475331980818886,
  26.329918234656684],
 'prec_str': '61.6/41.7/32.5/26.3',
 'ratio': 0.9680994221483417}